In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('train.csv')
df.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [4]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


## Handle missing values

In [6]:
df.dropna(inplace=True)
df.drop('id', axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221259 entries, 0 to 221258
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      221259 non-null  object 
 1   country   221259 non-null  object 
 2   store     221259 non-null  object 
 3   product   221259 non-null  object 
 4   num_sold  221259 non-null  float64
dtypes: float64(1), object(4)
memory usage: 8.4+ MB


In [7]:
df.describe()

,num_sold
count,221259.000000
mean,752.527382
std,690.165445
min,5.000000
25%,219.000000
50%,605.000000
75%,1114.000000
max,5939.000000


## Test Data

In [8]:
df_test = pd.read_csv('test.csv')
df_test.drop('id', axis=1, inplace=True)

## Quantify Data

In [9]:
# Train
df.date = df.date.str.split('-', expand=True)[0] + df.date.str.split('-', expand=True)[1] + df.date.str.split('-', expand=True)[2]
df.date = df.date.astype(int)

In [10]:
# Test
df_test.date = df_test.date.str.split('-', expand=True)[0] + df_test.date.str.split('-', expand=True)[1] + df_test.date.str.split('-', expand=True)[2]
df_test.date = df_test.date.astype(int)

In [11]:
df.head(3)

,date,country,store,product,num_sold
0,20100101,Canada,Discount Stickers,Kaggle,973.0
1,20100101,Canada,Discount Stickers,Kaggle Tiers,906.0
2,20100101,Canada,Discount Stickers,Kerneler,423.0


In [12]:
df['product'].value_counts().to_frame()

,count
product,
Kaggle,46026
Kaggle Tiers,46026
Kerneler Dark Mode,46025
Kerneler,45962
Holographic Goose,37220


In [13]:
df = df.join(pd.get_dummies(df['country'], prefix = 'country').astype(int)).drop('country', axis=1)
df = df.join(pd.get_dummies(df['store'], prefix = 'store').astype(int)).drop('store', axis=1)
df = df.join(pd.get_dummies(df['product'], prefix = 'product').astype(int)).drop('product', axis=1)

In [14]:
# Test
df_test = df_test.join(pd.get_dummies(df_test['country'], prefix = 'country').astype(int)).drop('country', axis=1)
df_test = df_test.join(pd.get_dummies(df_test['store'], prefix = 'store').astype(int)).drop('store', axis=1)
df_test = df_test.join(pd.get_dummies(df_test['product'], prefix = 'product').astype(int)).drop('product', axis=1)

In [15]:
# Correct dtypes
df['num_sold'] = df['num_sold'].astype(int)
df.head()

,date,num_sold,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
0,20100101,973,1,0,0,0,0,0,1,0,0,0,1,0,0,0
1,20100101,906,1,0,0,0,0,0,1,0,0,0,0,1,0,0
2,20100101,423,1,0,0,0,0,0,1,0,0,0,0,0,1,0
3,20100101,491,1,0,0,0,0,0,1,0,0,0,0,0,0,1
4,20100101,300,1,0,0,0,0,0,0,0,1,1,0,0,0,0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221259 entries, 0 to 221258
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype
---  ------                      --------------   -----
 0   date                        221259 non-null  int32
 1   num_sold                    221259 non-null  int32
 2   country_Canada              221259 non-null  int32
 3   country_Finland             221259 non-null  int32
 4   country_Italy               221259 non-null  int32
 5   country_Kenya               221259 non-null  int32
 6   country_Norway              221259 non-null  int32
 7   country_Singapore           221259 non-null  int32
 8   store_Discount Stickers     221259 non-null  int32
 9   store_Premium Sticker Mart  221259 non-null  int32
 10  store_Stickers for Less     221259 non-null  int32
 11  product_Holographic Goose   221259 non-null  int32
 12  product_Kaggle              221259 non-null  int32
 13  product_Kaggle Tiers        221259 non-null 

In [17]:
df.corr()['num_sold']

date                         -0.040564
num_sold                      1.000000
country_Canada                0.054136
country_Finland              -0.002843
country_Italy                -0.139527
country_Kenya                -0.449873
country_Norway                0.444573
country_Singapore             0.073334
store_Discount Stickers      -0.325233
store_Premium Sticker Mart    0.231209
store_Stickers for Less       0.089933
product_Holographic Goose    -0.361666
product_Kaggle                0.356331
product_Kaggle Tiers          0.197389
product_Kerneler             -0.145131
product_Kerneler Dark Mode   -0.075360
Name: num_sold, dtype: float64

## RF Model

In [18]:
X = df.drop('num_sold', axis=1)
y = df['num_sold']

X_test = df_test

In [19]:
scaler = MinMaxScaler()
X_tr = X
X_val = X[217259:221258]

X_tr.date = scaler.fit_transform(X_tr[['date']])
X_val.date = scaler.transform(X_val[['date']])
X_test.date = scaler.transform(X_test[['date']])

y_tr = y
y_val = y[217259:221258]

C:\Users\sajad\AppData\Local\Temp\ipykernel_1348\1496460746.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val.date = scaler.transform(X_val[['date']])


In [20]:
#rf = RandomForestRegressor(n_estimators=30, max_depth=15).fit(X_tr, y_tr)
#predictions = rf.predict(X_test)

#y_pred = rf.predict(X_val)
#mean_absolute_percentage_error(y_val, y_pred)

In [22]:
#def make_submission(prediction, sub_name):
#  my_submission = pd.DataFrame({'id':pd.read_csv('test.csv').id,'num_sold':prediction})
#  my_submission.to_csv('{}.csv'.format(sub_name),index=False)
#  print('A submission file has been made')

#make_submission(predictions.astype(int),'submission(rf)')

## NN

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=[15]),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

C:\Users\sajad\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\normalization\batch_normalization.py:142: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_percentage_error'])

In [26]:
hist = model.fit(X_tr, y_tr, epochs=10)

Epoch 1/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 191.5540 - mean_absolute_percentage_error: 73.0969
Epoch 2/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 104.2170 - mean_absolute_percentage_error: 16.5131
Epoch 3/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 96.1733 - mean_absolute_percentage_error: 14.5789
Epoch 4/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 92.8254 - mean_absolute_percentage_error: 13.8519
Epoch 5/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 90.0498 - mean_absolute_percentage_error: 13.5774
Epoch 6/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 89.5144 - mean_absolute_percentage_error: 13.5012
Epoch 7/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 88.7505 - mean_absolute_percentage_error: 13.3661
Epoch 8/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 88.2278 - mean_absolute_percentage_error: 13.2628
Epoch 9/10
6915/6915 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 87.7749 - mean_absolute_percent

In [27]:
predictions = model.predict(X_test)
predictions = predictions.reshape(98550,)

3080/3080 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [146]:
def make_submission(prediction, sub_name):
  my_submission = pd.DataFrame({'id':pd.read_csv('test.csv').id,'num_sold':prediction})
  my_submission.to_csv('{}.csv'.format(sub_name),index=False)
  print('A submission file has been made')

make_submission(predictions.astype(int),'submission(nn)')

A submission file has been made
